<a href="https://colab.research.google.com/github/geosensing/streetsense2/blob/main/get_streetview_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shapefile
import pandas as pd
import random
import numpy as np

from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt

In [3]:
# Load the shapefile (both .shp and .dbf should be in the same directory)
shp_path = "gadm41_IND_2.shp"  # Update path if needed
sf = shapefile.Reader(shp_path)

# Extract the fields and records
fields = [field[0] for field in sf.fields[1:]]  # Ignore first field ('DeletionFlag')
records = sf.records()

# Find the index of the "NAME_2" field (which contains "Mumbai City")
name_index = fields.index("NAME_2")

# Extract polylines for "NCT of Delhi"
delhi_shapes = [shape for record, shape in zip(records, sf.shapes()) if record[name_index] == "Mumbai City"]

# Get bounding box (min_x, min_y, max_x, max_y)
all_points = [point for shape in delhi_shapes for point in shape.points]
min_x = min(p[0] for p in all_points)
min_y = min(p[1] for p in all_points)
max_x = max(p[0] for p in all_points)
max_y = max(p[1] for p in all_points)

In [4]:
# Construct BBBike extract URL
bbbike_url = f"http://extract.bbbike.org/?sw_lng={min_x}&sw_lat={min_y}&ne_lng={max_x}&ne_lat={max_y}"
print("BBBike Extract URL:", bbbike_url)

BBBike Extract URL: http://extract.bbbike.org/?sw_lng=72.79152679500004&sw_lat=18.890695572000084&ne_lng=72.88465118300002&ne_lat=19.05473518399998


In [5]:
# Load the roads shapefile
roads_shp_path = "IND_IND.20_1+Mumbai City_roads.shp"  # Update path if needed
sf = shapefile.Reader(roads_shp_path)

# Extract fields and records
fields = [field[0] for field in sf.fields[1:]]  # Skip first field ('DeletionFlag')
records = sf.records()

# Inspect field names to find relevant ones
print("Fields in shapefile:", fields)

Fields in shapefile: ['osm_id', 'name', 'ref', 'type', 'oneway', 'bridge', 'maxspeed']


In [6]:
# Assume "road_type" or similar field exists; update this based on inspection
road_type_field = "type"  # Update if needed
road_type_index = fields.index(road_type_field)

unique_road_types = set(record[road_type_index] for record in sf.records())
print("Unique road types in shapefile:", unique_road_types)

Unique road types in shapefile: {'motorway_link', 'trunk', 'track', 'residential', 'pedestrian', 'living_street', 'footway', 'service', 'unclassified', 'proposed', 'services', 'path', 'motorway', 'secondary_link', 'trunk_link', 'primary_link', 'tertiary', 'construction', 'primary', 'tertiary_link', 'steps', 'secondary'}


In [7]:
# Extract field names
fields = [field[0] for field in sf.fields[1:]]  # Ignore DeletionFlag
print("Fields:", fields)  # Identify road type field

# Extract road data into a DataFrame
road_data = []
for record, shape in zip(sf.records(), sf.shapes()):
    if shape.shapeType == 3:  # Ensure it's a polyline
        start_lat, start_long = shape.points[0]
        end_lat, end_long = shape.points[-1]
        road_data.append(list(record) + [start_lat, start_long, end_lat, end_long])

# Create DataFrame
df_roads = pd.DataFrame(road_data, columns=fields + ["start_lat", "start_long", "end_lat", "end_long"])

# Display sample
df_roads.head()

Fields: ['osm_id', 'name', 'ref', 'type', 'oneway', 'bridge', 'maxspeed']


,osm_id,name,ref,type,oneway,bridge,maxspeed,start_lat,start_long,end_lat,end_long
0,22845315,Badaruddin Tayabji Marg,,tertiary,0,0,NaN,72.830680,18.942550,72.832707,18.944611
1,22845318,Homi Modi Street,,residential,1,0,NaN,72.831716,18.931634,72.834059,18.931166
2,22845319,Bastion Road,,unclassified,1,0,NaN,72.832398,18.934659,72.833990,18.938778
3,22845328,M K Amin Marg,,service,1,0,NaN,72.835350,18.937352,72.835133,18.937306
4,22845331,Jeejeebhoy Dadabhoy Lane,,residential,0,0,NaN,72.835407,18.936243,72.833872,18.936409


In [8]:
def haversine(lat1, lon1, lat2, lon2):
    """Compute the Haversine distance (in meters) between two lat/lon points."""
    R = 6371000  # Earth's radius in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2.0)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # Distance in meters

In [10]:
df_roads["LENGTH_M"] = df_roads.apply(
    lambda row: haversine(row["start_lat"], row["start_long"], row["end_lat"], row["end_long"]),
    axis=1
)

# Check length distribution
print(df_roads["LENGTH_M"].describe())

count    9883.000000
mean       85.548524
std       130.741312
min         0.000000
25%        17.811029
50%        46.735954
75%       103.790353
max      3532.774422
Name: LENGTH_M, dtype: float64


In [11]:
def split_road_segment(row, segment_length=500):
    """Splits a road segment into equal chunks of segment_length (in meters)."""
    road_length = row["LENGTH_M"]

    if road_length <= segment_length:
        return [row.to_dict()]  # Return original row as a list to avoid NaNs

    num_segments = int(np.ceil(road_length / segment_length))  # Number of 500m chunks

    start_lat, start_long = row["start_lat"], row["start_long"]
    end_lat, end_long = row["end_lat"], row["end_long"]

    lat_step = (end_lat - start_lat) / num_segments
    long_step = (end_long - start_long) / num_segments

    split_segments = []
    for i in range(num_segments):
        new_start_lat = start_lat + (i * lat_step)
        new_start_long = start_long + (i * long_step)
        new_end_lat = new_start_lat + lat_step
        new_end_long = new_start_long + long_step

        new_segment = row.to_dict()
        new_segment["start_lat"] = new_start_lat
        new_segment["start_long"] = new_start_long
        new_segment["end_lat"] = new_end_lat
        new_segment["end_long"] = new_end_long
        new_segment["SEGMENT_PART"] = i + 1
        new_segment["LENGTH_M"] = segment_length  # Set length to 500m

        split_segments.append(new_segment)

    return split_segments

In [12]:
df_long_roads = df_roads[df_roads["LENGTH_M"] > 500]
df_short_roads = df_roads[df_roads["LENGTH_M"] <= 500]

# Apply fixed chunking function
df_long_roads_split = df_long_roads.apply(split_road_segment, axis=1)

# Ensure list format before explode()
df_long_roads_split = df_long_roads_split.explode().reset_index(drop=True)

# Convert dictionaries back to DataFrame
df_long_roads_split = pd.DataFrame(list(df_long_roads_split))

# Combine back
df_roads_final = pd.concat([df_short_roads, df_long_roads_split], ignore_index=True)

print("✅ Roads successfully chunked into 500m segments!")

✅ Roads successfully chunked into 500m segments!


In [13]:
# Randomly sample 1000 roads
df_sampled = df_roads_final.sample(n=1000, random_state=42)  # Change `n` if needed

df_sampled.head()

,osm_id,name,ref,type,oneway,bridge,maxspeed,start_lat,start_long,end_lat,end_long,LENGTH_M,SEGMENT_PART
8776,1241393020,Maharshi Karve Road,,secondary,1,0,NaN,72.826021,18.942294,72.824820,18.944545,152.643014,NaN
2199,113715149,Shankar Rao H. Parelkar Marg,,residential,0,0,NaN,72.836113,19.023993,72.837444,19.024426,148.660302,NaN
9596,1343390136,,,residential,0,0,NaN,72.862519,19.029469,72.862506,19.029746,9.173652,NaN
8490,1236044645,Dr Annie Besant Road,,primary,1,0,NaN,72.818250,19.006266,72.816708,19.004104,185.632231,NaN
3314,370666906,,,service,0,0,NaN,72.819879,18.993948,72.820510,18.995602,88.690588,NaN


In [14]:
import pandas as pd
import numpy as np
import requests
import os
from urllib.parse import urlencode
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import random

class StreetViewImageFetcher:
    """
    A class to fetch Google Street View images from road segments in a pandas DataFrame.
    Uses both start and end coordinates to sample points along road segments.
    """

    def __init__(self, api_key, image_size="600x400", fov=90, output_dir="street_view_images"):
        """
        Initialize the Street View Image Fetcher.

        Args:
            api_key (str): Your Google API key with Street View Static API enabled
            image_size (str): Size of the images to fetch (width x height)
            fov (int): Field of view (zoom level)
            output_dir (str): Directory to save the images
        """
        self.api_key = api_key
        self.image_size = image_size
        self.fov = fov
        self.output_dir = output_dir

        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

    def get_random_heading(self):
        """Generate a random heading (compass direction) between 0-360 degrees."""
        return random.randint(0, 359)

    def get_random_pitch(self):
        """
        Generate a random pitch (vertical angle) mostly between -30 and 30 degrees.
        More weighted towards horizon view (0) with some variation.
        """
        return int((random.random() - 0.5) * 60)

    def interpolate_points(self, start_lat, start_lng, end_lat, end_lng, num_points=3):
        """
        Interpolate points along a line between start and end coordinates.

        Args:
            start_lat (float): Starting latitude
            start_lng (float): Starting longitude
            end_lat (float): Ending latitude
            end_lng (float): Ending longitude
            num_points (int): Number of points to sample (including start and end)

        Returns:
            list: List of (lat, lng) tuples
        """
        if num_points < 2:
            return [(start_lat, start_lng)]

        points = []
        for i in range(num_points):
            # Calculate the fraction of the way from start to end
            fraction = i / (num_points - 1)

            # Linear interpolation
            lat = start_lat + fraction * (end_lat - start_lat)
            lng = start_lng + fraction * (end_lng - start_lng)

            points.append((lat, lng))

        return points

    def build_street_view_url(self, lat, lng, heading, pitch):
        """
        Build a Google Street View image URL for a given location and angles.

        Args:
            lat (float): Latitude coordinate
            lng (float): Longitude coordinate
            heading (int): Compass direction (0-360)
            pitch (int): Vertical angle (-90 to 90)

        Returns:
            str: Complete URL to fetch the Street View image
        """
        params = {
            'size': self.image_size,
            'location': f"{lat},{lng}",
            'heading': heading,
            'pitch': pitch,
            'fov': self.fov,
            'key': self.api_key
        }

        return f"https://maps.googleapis.com/maps/api/streetview?{urlencode(params)}"

    def fetch_image(self, url, filename):
        """
        Fetch an image from a URL and save it to disk.

        Args:
            url (str): URL to fetch the image from
            filename (str): Path to save the image to

        Returns:
            bool: True if the image was successfully fetched, False otherwise
        """
        try:
            response = requests.get(url, timeout=10)

            # Check if we got a valid image (not the "no image" placeholder)
            if response.status_code == 200 and len(response.content) > 5000:
                with open(filename, 'wb') as f:
                    f.write(response.content)
                return True
            return False
        except Exception as e:
            print(f"Error fetching image: {e}")
            return False

    def process_road_segment(self, idx, row, start_lat_col='start_lat', start_lng_col='start_long',
                            end_lat_col='end_lat', end_lng_col='end_long',
                            points_per_segment=3, images_per_point=2):
        """
        Process a single road segment (row in DataFrame).

        Args:
            idx (int): Index of the row
            row (pandas.Series): Row from the DataFrame
            start_lat_col (str): Name of the starting latitude column
            start_lng_col (str): Name of the starting longitude column
            end_lat_col (str): Name of the ending latitude column
            end_lng_col (str): Name of the ending longitude column
            points_per_segment (int): Number of points to sample along the segment
            images_per_point (int): Number of random angle images to generate per point

        Returns:
            list: Information about the processed images
        """
        results = []

        # Get start and end coordinates
        start_lat = row[start_lat_col]
        start_lng = row[start_lng_col]
        end_lat = row[end_lat_col]
        end_lng = row[end_lng_col]

        # Interpolate points along the road segment
        points = self.interpolate_points(start_lat, start_lng, end_lat, end_lng, points_per_segment)

        for point_idx, (lat, lng) in enumerate(points):
            for img_idx in range(images_per_point):
                heading = self.get_random_heading()
                pitch = self.get_random_pitch()

                url = self.build_street_view_url(lat, lng, heading, pitch)
                filename = os.path.join(
                    self.output_dir,
                    f"segment_{idx}_point_{point_idx}_img_{img_idx}_h{heading}_p{pitch}.jpg"
                )

                success = self.fetch_image(url, filename)

                result = {
                    'segment_index': idx,
                    'point_index': point_idx,
                    'image_index': img_idx,
                    'lat': lat,
                    'lng': lng,
                    'heading': heading,
                    'pitch': pitch,
                    'filename': filename if success else None,
                    'success': success
                }

                # Add any additional row data if available
                for col in row.index:
                    if col not in [start_lat_col, start_lng_col, end_lat_col, end_lng_col]:
                        result[col] = row[col]

                results.append(result)

                # Add a small delay to avoid hitting API rate limits
                time.sleep(0.2)

        return results

    def fetch_images_from_road_segments(self, df, start_lat_col='start_lat', start_lng_col='start_long',
                                      end_lat_col='end_lat', end_lng_col='end_long',
                                      points_per_segment=3, images_per_point=2, max_workers=5):
        """
        Fetch Street View images for all road segments in a DataFrame.

        Args:
            df (pandas.DataFrame): DataFrame containing road segment coordinates
            start_lat_col (str): Name of the starting latitude column
            start_lng_col (str): Name of the starting longitude column
            end_lat_col (str): Name of the ending latitude column
            end_lng_col (str): Name of the ending longitude column
            points_per_segment (int): Number of points to sample along each segment
            images_per_point (int): Number of random angle images to generate per point
            max_workers (int): Maximum number of parallel workers to use

        Returns:
            pandas.DataFrame: DataFrame with information about all processed images
        """
        all_results = []

        print(f"Fetching Street View images for {len(df)} road segments...")

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []

            for idx, row in df.iterrows():
                future = executor.submit(
                    self.process_road_segment, idx, row, start_lat_col, start_lng_col,
                    end_lat_col, end_lng_col, points_per_segment, images_per_point
                )
                futures.append(future)

            # Process results as they complete
            for future in tqdm(futures, total=len(futures), desc="Processing road segments"):
                results = future.result()
                all_results.extend(results)

        # Convert results to DataFrame
        results_df = pd.DataFrame(all_results)

        # Print summary
        success_count = results_df['success'].sum()
        total_count = len(results_df)
        print(f"Successfully fetched {success_count} of {total_count} images "
              f"({success_count/total_count*100:.1f}%)")

        return results_df


# Example usage
def fetch_street_view_images(roads_df, api_key, output_dir="street_view_images", points_per_segment=2, images_per_point=1):
    """
    Main function to fetch Street View images from road segments.

    Args:
        roads_df (pandas.DataFrame): DataFrame containing road segments with start/end coordinates
        api_key (str): Google API key with Street View Static API enabled
        output_dir (str): Directory to save images
        points_per_segment (int): Number of points to sample along each segment
        images_per_point (int): Number of images with different angles at each point

    Returns:
        pandas.DataFrame: Results with image details
    """
    # Initialize the fetcher
    fetcher = StreetViewImageFetcher(
        api_key=api_key,
        image_size="600x400",
        output_dir=output_dir
    )

    # Fetch images for all road segments
    results = fetcher.fetch_images_from_road_segments(
        df=roads_df,
        start_lat_col='start_lat',
        start_lng_col='start_long',
        end_lat_col='end_lat',
        end_lng_col='end_long',
        points_per_segment=points_per_segment,
        images_per_point=images_per_point,
        max_workers=3
    )

    # Save results to CSV
    results.to_csv(os.path.join(output_dir, 'street_view_results.csv'), index=False)

    print(f"Done! Images saved to '{output_dir}' and results saved to '{output_dir}/street_view_results.csv'")

    return results

In [15]:
# Your Google API key
API_KEY = ""

# Fetch Street View images
results = fetch_street_view_images(
    roads_df=df_sampled,
    api_key=API_KEY,
    points_per_segment=3,  # Sample 3 points along each road segment
    images_per_point=2     # Take 2 images with different angles at each point
)

Fetching Street View images for 1000 road segments...


Processing road segments:   3%|▎         | 33/1000 [00:16<08:09,  1.97it/s]


KeyboardInterrupt: 